In [1]:
import caveclient
import pandas as pd
import numpy as np
import os
import tqdm
import standard_transform

from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib


trafo_streamline = standard_transform.datasets.v1dd_streamline_nm()

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)

mat_version = 1196

HOME = os.path.expanduser("~")

# NOTE: adjust for your system
# data_dir = f"{HOME}/SWDB_2025_Connectomics/data/"
data_dir = f"{HOME}/SWDB_2025_Connectomics/data/"

client = caveclient.CAVEclient(
    "v1dd",
    server_address="https://global.em.brain.allentech.org",
    version=mat_version,
)

/Users/sven.dorkenwald/anaconda3/envs/chain-py312/lib/python3.12/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Proofread information


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Proofreading and data quality

Understanding this variablity in data quality is critical when interpretting electron microscopy reconstructions.

Automated segmentation of neuronal processes in dense EM imaging is challenging at the size of entire neurons, which can have millimeters of axons and dendrites. The automated segmentation algorithms used in the EM data for this project are not perfect, and so proofreading is necessary to obtain accurate reconstructions of a cell and confidence in the connectivity

Axon and dendrite compartment status are marked separately, as proofreading effort was applied differently to the different compartments in some cells.  In all cases, a status of `TRUE` indicates that false merges have been comprehensively removed, and the compartment is at least ‘clean’. Consult the ‘strategy’ column if completeness of the compartment is relevant to your  research.

Some cells were extended to different degrees of completeness, or with different research goals in mind. This is denoted by 'strategy_axon', which may be one of:

<ul>
    <li>none: No cleaning, and no extension, and status is `FALSE`. </li>
    <li>axon_partially_extended: The axon was extended outward from the soma, following each branch to its termination. Output synapses represent a sampling of potential partners. </li>
    <li>axon_interareal: The axon was extended with a preference for branches that projected to other brain areas. Some axon branches were fully extended, but local connections may be incomplete. Output synapses represent a sampling of potential partners. </li>
    <li>axon_fully_extended: Axon was extended outward from the soma, following each branch to its termination. After initial extension, every endpoint was identified, manually inspected, and extended again if possible. Output synapses represent a largely complete sampling of partners.. </li>
</ul>

<b> For this workshop, we treat all cells with at least `axon_partially_extended` as equally trustworth.</b> This may not be a safe assumption for all analysis, and we are happy to provide more guidance depending on the research question.
    
</div>

In [2]:
neuron_soma_df = client.materialize.query_table("neurons_soma_model")
axon_proof_df = client.materialize.query_table(
    "proofreading_status_and_strategy",
    filter_in_dict={"strategy_axon": ["axon_fully_extended"]},
)
single_soma_df = client.materialize.query_view(
    "single_somas", split_positions=True, desired_resolution=[1, 1, 1]
)

dendrite_proof_root_ids = np.array(single_soma_df["pt_root_id"])
dendrite_proof_root_ids = dendrite_proof_root_ids[
    np.isin(dendrite_proof_root_ids, neuron_soma_df["pt_root_id"])
]
axon_proof_root_ids = np.array(axon_proof_df["pt_root_id"])

In [3]:
np.save(f"{data_dir}/proofread_axon_list_{mat_version}.npy", axon_proof_root_ids)
np.save(
    f"{data_dir}/proofread_dendrite_list_{mat_version}.npy", dendrite_proof_root_ids
)

## Cell information


In [4]:
cell_type_df = client.materialize.query_table(
    "cell_type_snds", desired_resolution=[1, 1, 1], split_positions=True
).rename(columns={"classification_system": "cell_type_coarse"})
cell_type_df = cell_type_df[
    ~cell_type_df["cell_type"].map(
        lambda x: x.startswith("L23_")
        or x.startswith("L1_inh")
        or x.startswith("L6_6b")
    )
]
nuc_df = client.materialize.query_view(
    "nucleus_alternative_lookup", split_positions=True, desired_resolution=[1, 1, 1]
)

reference_point = np.array([900_000.0, 700_000.0, 300_000.0])
nuc_df[["pt_position_trform_x", "pt_position_trform_y", "pt_position_trform_z"]] = trafo_streamline.radial_points(reference_point, nuc_df[["pt_position_x", "pt_position_y", "pt_position_z"]]) * 1000

In [5]:
def map_cell_type(ct_str):
    if ct_str.startswith("L"):
        return "-".join(ct_str.split("_")[:2])
    else:
        return ct_str[:3]


cell_type_df["cell_type"] = cell_type_df["cell_type"].map(map_cell_type)
cell_type_df["cell_type_coarse"] = cell_type_df["cell_type_coarse"].map(
    lambda x: {"inh": "I", "exc": "E"}[x]
)

In [6]:
soma_ct_df = pd.merge(
    nuc_df[
        [
            "id",
            "pt_position_x",
            "pt_position_y",
            "pt_position_z",
            "pt_position_trform_x",
            "pt_position_trform_y",
            "pt_position_trform_z",
            "pt_root_id",
            "volume",
        ]
    ],
    cell_type_df[["pt_root_id", "cell_type_coarse", "cell_type"]],
    how="left",
    on="pt_root_id",
)

In [7]:
soma_ct_df

,id,pt_position_x,pt_position_y,pt_position_z,pt_position_trform_x,pt_position_trform_y,pt_position_trform_z,pt_root_id,volume,cell_type_coarse,cell_type
0,228132,632828,749849,738270,-323721.447979,549910.283106,392909.832613,864691132737039043,458.464831,NaN,NaN
1,543247,1304922,977915,83880,330339.020171,595962.275760,-306424.551354,864691132730839988,73.345940,NaN,NaN
2,203262,624680,531094,283770,-252082.627894,203770.728235,21544.029756,864691132654552792,338.276613,E,L3-IT
3,350562,894573,478559,163530,20989.259196,117514.626427,-98554.035375,864691132773514104,326.965400,E,L2-IT
4,718122,1729859,674111,781200,803635.726721,475075.415268,467669.881328,864691132774106773,333.888647,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
207450,527607,1262940,628094,734445,352037.266857,418290.963101,436647.625407,864691132639606383,100.547645,NaN,NaN
207451,168582,491518,1057067,92070,-521847.319737,688587.870919,-335347.345410,864691133042980384,369.919126,NaN,NaN
207452,29422,302330,415005,81855,-570524.837595,35550.648588,-177610.741633,0,285.031368,NaN,NaN
207453,422767,1065603,538932,36405,191665.101812,141490.682021,-231412.518393,864691132851361283,394.724290,NaN,NaN


In [8]:
soma_ct_df.to_feather(
    f"{data_dir}/soma_and_cell_type_{mat_version}.feather", compression="zstd"
)

## Synapse information


In [9]:
syn_path = f"{data_dir}/syn_df_all_to_proofread_to_all_{mat_version}.feather"

if os.path.exists(syn_path):
    syn_df = pd.read_feather(syn_path)
else:
    syn_dfs = []

    pre_syn_path = (
        f"{data_dir}/pre_syn_df_all_to_proofread_to_all_{mat_version}.feather"
    )
    if os.path.exists(pre_syn_path):
        pre_syn_df = pd.read_feather(pre_syn_path)
    else:
        for pre_root_ids in tqdm.tqdm(
            np.array_split(axon_proof_root_ids, len(axon_proof_root_ids) // 40)
        ):
            syn_df_chunk = client.materialize.query_table(
                "synapses_v1dd",
                filter_in_dict={"pre_pt_root_id": pre_root_ids},
                desired_resolution=[1, 1, 1],
                split_positions=True,
            )
            syn_df_chunk = syn_df_chunk[
                syn_df_chunk["pre_pt_root_id"] != syn_df_chunk["post_pt_root_id"]
            ]
            syn_df_chunk = syn_df_chunk.drop(
                columns=[
                    "created",
                    "valid",
                    "superceded_id",
                    "pre_pt_supervoxel_id",
                    "post_pt_supervoxel_id",
                ]
            )
            assert len(syn_df_chunk) < 500_000
            syn_dfs.append(syn_df_chunk)
        pre_syn_df = pd.concat(syn_dfs)
        pre_syn_df.to_feather(syn_path, compression="zstd")

    post_syn_path = (
        f"{data_dir}/post_syn_df_all_to_proofread_to_all_{mat_version}.feather"
    )
    if os.path.exists(post_syn_path):
        post_syn_df = pd.read_feather(post_syn_path)
    else:
        for post_root_ids in tqdm.tqdm(
            np.array_split(axon_proof_root_ids, len(axon_proof_root_ids) // 40)
        ):
            syn_df_chunk = client.materialize.query_table(
                "synapses_v1dd",
                filter_in_dict={"post_pt_root_id": post_root_ids},
                desired_resolution=[1, 1, 1],
                split_positions=True,
            )
            syn_df_chunk = syn_df_chunk[
                syn_df_chunk["pre_pt_root_id"] != syn_df_chunk["post_pt_root_id"]
            ]
            syn_df_chunk = syn_df_chunk.drop(
                columns=[
                    "created",
                    "valid",
                    "superceded_id",
                    "pre_pt_supervoxel_id",
                    "post_pt_supervoxel_id",
                ]
            )
            assert len(syn_df_chunk) < 500_000
            syn_dfs.append(syn_df_chunk)
        post_syn_df = pd.concat(syn_dfs)
        post_syn_df.to_feather(syn_path, compression="zstd")

    syn_df = pd.concat([pre_syn_df, post_syn_df])
    syn_df = syn_df.drop_duplicates("id", keep="first").reset_index(drop=True)
    syn_df.to_feather(syn_path, compression="zstd")

100%|███████████████████████████████████████████| 30/30 [35:47<00:00, 71.58s/it]


In [10]:
syn_df

,id,pre_pt_position_x,pre_pt_position_y,pre_pt_position_z,post_pt_position_x,post_pt_position_y,post_pt_position_z,ctr_pt_position_x,ctr_pt_position_y,ctr_pt_position_z,size,pre_pt_root_id,post_pt_root_id
0,354386968,758200.5,802316.1,304380.0,757861.0,802558.6,304650.0,757967.7,802597.4,304380.0,240,864691132536286810,864691132734919083
1,378070488,792063.2,514342.5,183735.0,792664.6,514284.3,183915.0,792412.4,514294.0,183735.0,3056,864691132572190492,864691132606767301
2,499493001,977071.3,390075.8,191340.0,976974.3,390104.9,190935.0,976838.5,390337.7,190935.0,1346,864691132573738810,864691132747578447
3,119675985,444260.0,602544.6,3285.0,443988.4,602311.8,3555.0,444182.4,602370.0,3780.0,3637,864691132572564252,864691132654028028
4,220616943,574501.9,337249.6,258570.0,574152.7,337016.8,258570.0,574337.0,336900.4,258570.0,420,864691132558380553,864691132828255906
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8204492,489487534,952307.2,817166.8,278235.0,952113.2,817428.7,277830.0,952326.6,817341.4,278055.0,1466,864691132640842277,864691132831272057
8204493,489987803,959863.5,698244.8,260730.0,960076.9,697837.4,260910.0,959834.4,697973.2,260820.0,745,864691132731252911,864691132831272057
8204494,465370688,920500.9,767415.5,140175.0,920200.2,767890.8,139635.0,920549.4,767764.7,140040.0,290,864691132603059809,864691132831272057
8204495,450289242,902575.3,759607.0,153045.0,902914.8,760043.5,152460.0,902895.4,759742.8,152730.0,537,864691130711898993,864691132831272057


In [11]:
syn_label_path = (
    f"{data_dir}/syn_label_df_all_to_proofread_to_all_{mat_version}.feather"
)


def get_syn_label_chunk(syn_id_chunk) -> pd.DataFrame:
    syn_label_chunk = client.materialize.query_table(
        "synapse_target_predictions_ssa",
        filter_in_dict={"target_id": syn_id_chunk},
        log_warning=False,
        merge_reference=False,
        select_columns=["target_id", "tag"],
    )
    syn_label_chunk = syn_label_chunk.rename(columns={"target_id": "id"})
    return syn_label_chunk


if os.path.exists(syn_label_path):
    syn_label_df = pd.read_feather(syn_label_path)
else:
    syn_ids = syn_df["id"].values
    chunk_size = 25_000
    n_chunks = len(syn_ids) // chunk_size + 1

    syn_id_chunks = np.array_split(syn_ids, n_chunks)

    with tqdm_joblib(desc="Pulling synapse labels", total=len(syn_id_chunks)):
        syn_label_chunks: list[pd.DataFrame] = Parallel(n_jobs=-2)(
            delayed(get_syn_label_chunk)(syn_id_chunk) for syn_id_chunk in syn_id_chunks
        )

    syn_label_df: pd.DataFrame = pd.concat(syn_label_chunks).set_index("id")
    syn_label_df.to_feather(
        syn_label_path,
        compression="zstd",
    )

syn_label_df

Pulling synapse labels:   0%|          | 0/329 [00:00<?, ?it/s]

,tag
id,
35903092,spine
51716092,spine
59062330,spine
62437192,shaft
62841618,spine
...,...
698454520,spine
732146976,shaft
733840618,spine
